# 1. Experimenting with a node endpoint
In this first example, we will explore how to access data from an Ethereum node.

Please create a free account with [Moralis.io](https://admin.moralis.io/register), and get your own endpoint URL. You may be able to use this notebook endpoint, but you may run into rate limits if other participants are using it at the same time. Or at a later point in time this endpoint may not work at all anymore.

In [1]:
endpoint = "https://speedy-nodes-nyc.moralis.io/03c966587b022c980f59136b/eth/mainnet/archive"

## 1.1 Raw requests
At first, we'll be making a raw request and get a JSON response about a single block and its transactions.

In [2]:
import json
import requests

In [3]:
def create_request(method, params):
    payload = json.dumps({"method": method, "params": params, "id":1})
    headers = {'content-type': "application/json"}
    try:
        response = requests.request("POST", endpoint, data=payload, headers=headers)
        return(json.loads(response.text))
    except requests.exceptions.RequestException as e:
        print(e)
    except:
        print('No response from endpoint!')

In [4]:
blockNumber = 10000000
response = create_request(method="eth_getBlockByNumber", params=[hex(blockNumber),True])

## 1.2 A block JSON
Lets look at the raw JSON output of this request.
It consists of block metadata, and a list of transactions that are contained in the block.

In [5]:
blockWithTX = response["result"]
blockWithTX

{'difficulty': '0x8792c6f47f70f',
 'extraData': '0x505059452d65746865726d696e652d61736961312d31',
 'gasLimit': '0x98705c',
 'gasUsed': '0x9824b3',
 'hash': '0xaa20f7bde5be60603f11a45fc4923aab7552be775403fc00c2e6b805e6297dbe',
 'logsBloom': '0x886480c00200620d84180d0470000c503081160044d050158080037401107060120040105281100100104500414203040a2080034814200610da1208a638d16e440c024880800301e1004c2b022850602000084c3249a0c084569c90c2002001586241041e8004035a4400a0100938001e041180083180b0340661372060401428c020087410402b9484028100049481900c08034864314688d001548c3000828e542284180280006402a28a0264da00ac22300400620960983206603200084040122a4739080501251542082020a4087c000281c08800898d0900024047380000127038098e0908010800004290c84201661040200201c0004b8490ad588804',
 'miner': '0xea674fdde714fd979de3edf0f56aa9716b898ec8',
 'mixHash': '0x37fde31175fe180346444d15b4dfc6a9da3b2b41ee2298ceeccaf888b2d45df4',
 'nonce': '0x2f6923f80426f157',
 'number': '0x989680',
 'parentHash': '0x966bf6849da92ff2a0e3db9a371f5b9

In [6]:
len(blockWithTX["transactions"]) # How many transactions are contained in this block?

103

## 1.3 A transaction JSON

In [7]:
blockWithTX["transactions"][0]

{'blockHash': '0xaa20f7bde5be60603f11a45fc4923aab7552be775403fc00c2e6b805e6297dbe',
 'blockNumber': '0x989680',
 'from': '0xea674fdde714fd979de3edf0f56aa9716b898ec8',
 'gas': '0xc350',
 'gasPrice': '0x3b9aca00',
 'hash': '0x4a1e3e3a2aa4aa79a777d0ae3e2c3a6de158226134123f6c14334964c6ec70cf',
 'input': '0x',
 'nonce': '0x18bc14e',
 'to': '0x60f18d941f6253e3f7082ea0db3bc3944e7e9d40',
 'transactionIndex': '0x0',
 'value': '0x554b81ec144eea9',
 'type': '0x0',
 'v': '0x26',
 'r': '0xbf3209563b738f102d851c78ff690ab9a10a4c71daacfcc75119acdd7e6231a3',
 's': '0x247b41729d11c3496a99deb8d84b19cdb563655b7801ea9fc4dcdbe814178a1a'}

In [8]:
#  r and s are outputs of an ECDSA signature, and v is the recovery id.
# https://bitcoin.stackexchange.com/questions/38351/ecdsa-v-r-s-what-is-v

## 1.4 Using web3py
At this point it's becoming clear that these json results contain mostly hexadecimal values, which is unwieldy.

Let's use web3py to have these values automatically converted to integers where applicable. This has the additional benefit that we can write less code.

In [9]:
from web3 import Web3

In [10]:
w3 = Web3(Web3.HTTPProvider(endpoint))
w3.isConnected()

True

In [11]:
blockWithTX = w3.eth.get_block(hex(blockNumber), full_transactions=True)
blockWithTX

AttributeDict({'difficulty': 2385031566194447,
 'extraData': HexBytes('0x505059452d65746865726d696e652d61736961312d31'),
 'gasLimit': 9990236,
 'gasUsed': 9970867,
 'hash': HexBytes('0xaa20f7bde5be60603f11a45fc4923aab7552be775403fc00c2e6b805e6297dbe'),
 'logsBloom': HexBytes('0x886480c00200620d84180d0470000c503081160044d050158080037401107060120040105281100100104500414203040a2080034814200610da1208a638d16e440c024880800301e1004c2b022850602000084c3249a0c084569c90c2002001586241041e8004035a4400a0100938001e041180083180b0340661372060401428c020087410402b9484028100049481900c08034864314688d001548c3000828e542284180280006402a28a0264da00ac22300400620960983206603200084040122a4739080501251542082020a4087c000281c08800898d0900024047380000127038098e0908010800004290c84201661040200201c0004b8490ad588804'),
 'miner': '0xEA674fdDe714fd979de3EdF0F56AA9716B898ec8',
 'mixHash': HexBytes('0x37fde31175fe180346444d15b4dfc6a9da3b2b41ee2298ceeccaf888b2d45df4'),
 'nonce': HexBytes('0x2f6923f80426f157'),
 'number': 1000

In [12]:
blockWithTX.transactions[0]

AttributeDict({'blockHash': HexBytes('0xaa20f7bde5be60603f11a45fc4923aab7552be775403fc00c2e6b805e6297dbe'),
 'blockNumber': 10000000,
 'from': '0xEA674fdDe714fd979de3EdF0F56AA9716B898ec8',
 'gas': 50000,
 'gasPrice': 1000000000,
 'hash': HexBytes('0x4a1e3e3a2aa4aa79a777d0ae3e2c3a6de158226134123f6c14334964c6ec70cf'),
 'input': '0x',
 'nonce': 25936206,
 'to': '0x60f18d941F6253E3F7082Ea0db3bc3944e7e9D40',
 'transactionIndex': 0,
 'value': 384134310464384681,
 'type': '0x0',
 'v': 38,
 'r': HexBytes('0xbf3209563b738f102d851c78ff690ab9a10a4c71daacfcc75119acdd7e6231a3'),
 's': HexBytes('0x247b41729d11c3496a99deb8d84b19cdb563655b7801ea9fc4dcdbe814178a1a')})

Notice how the **from** and **to** addresses are now partially uppercase?
These are the checksummed versions of the lowercase addreses. This case-sensivity is used for checksum validation. The address is compared against the raw binary keccak-256 hash of the address bytes, and where there are letters in the same corresponding place as a "1" bit, the letter is capitalized. Just make sure you either always use checksum addreses or lowercase addresses. Otherwise your analysis might perceive the same address as two different ones!

## 1.5 Let's turn the transaction JSON into a dataframe

In [13]:
import pandas as pd
transactionDF = pd.DataFrame(blockWithTX.transactions)[["from", "to", "value"]]
transactionDF

,from,to,value
0,0xEA674fdDe714fd979de3EdF0F56AA9716B898ec8,0x60f18d941F6253E3F7082Ea0db3bc3944e7e9D40,384134310464384681
1,0xCAc725beF4f114F728cbCfd744a731C2a463c3Fc,0x54A9AC972aB091f0ddBb051317F02882e70885C0,195000000000000000
2,0xFB90501083a3b6AF766c8dA35d3Dde01eB0d2a68,0x2684CA06FFE6AC41a3137F1Ed5c87bb35E7fD82A,69056900000000000
3,0x876EabF441B2EE5B5b0554Fd502a8E0600950cFa,0x8A9bF82B9df4594291fceEFe6bC4835186f599b2,51389774580000000000
4,0x876EabF441B2EE5B5b0554Fd502a8E0600950cFa,0x37b5AB65Da648146b4C5EA955a1Ec37671FE6B68,5126333960000000000
...,...,...,...
98,0x8b6FD247c940D1a8b44BAeB918Db74195F956cc1,0xf93501385BdE9305C6a36E13D0Ee499B01574338,35000000000000000
99,0x9C43D6a63F7FD0AE469Ec0aeda2a90Be93038F59,0xdAC17F958D2ee523a2206206994597C13D831ec7,0
100,0x2d0966cFc198190f37179f072C63f464f36c7da5,0xf8A4d3a0B5859a24cd1320BA014ab17F623612e2,0
101,0x2d0966cFc198190f37179f072C63f464f36c7da5,0xf8A4d3a0B5859a24cd1320BA014ab17F623612e2,0


You can already see that the value column seems to contain very large numbers. This is because they are in the unit **Wei**. One Ether corresponds to 10^18 Wei. So we can convert the column accordingly.

In [14]:
transactionDF["value"] = transactionDF["value"].apply(lambda x: x/10**18)

In [15]:
transactionDF

,from,to,value
0,0xEA674fdDe714fd979de3EdF0F56AA9716B898ec8,0x60f18d941F6253E3F7082Ea0db3bc3944e7e9D40,0.384134
1,0xCAc725beF4f114F728cbCfd744a731C2a463c3Fc,0x54A9AC972aB091f0ddBb051317F02882e70885C0,0.195000
2,0xFB90501083a3b6AF766c8dA35d3Dde01eB0d2a68,0x2684CA06FFE6AC41a3137F1Ed5c87bb35E7fD82A,0.069057
3,0x876EabF441B2EE5B5b0554Fd502a8E0600950cFa,0x8A9bF82B9df4594291fceEFe6bC4835186f599b2,51.389775
4,0x876EabF441B2EE5B5b0554Fd502a8E0600950cFa,0x37b5AB65Da648146b4C5EA955a1Ec37671FE6B68,5.126334
...,...,...,...
98,0x8b6FD247c940D1a8b44BAeB918Db74195F956cc1,0xf93501385BdE9305C6a36E13D0Ee499B01574338,0.035000
99,0x9C43D6a63F7FD0AE469Ec0aeda2a90Be93038F59,0xdAC17F958D2ee523a2206206994597C13D831ec7,0.000000
100,0x2d0966cFc198190f37179f072C63f464f36c7da5,0xf8A4d3a0B5859a24cd1320BA014ab17F623612e2,0.000000
101,0x2d0966cFc198190f37179f072C63f464f36c7da5,0xf8A4d3a0B5859a24cd1320BA014ab17F623612e2,0.000000
